In [1]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential

from keras.utils import to_categorical
from keras.optimizers import RMSprop, Adam

from keras.applications import xception, inception_v3

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

Using TensorFlow backend.


In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'

batch_size = 16
target_size=(299, 299)

def add_preprocess(base_model, preprocess_func, inputs_shape=(299, 299, 3)):
    inputs = Input(shape=inputs_shape)
    x = Lambda(preprocess_func)(inputs)
    outputs = base_model(x)
    model = Model(inputs, outputs)
    return model

### train

emsemble a few augmentation training data

In [3]:
base_model_x = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
model_x = add_preprocess(base_model_x, xception.preprocess_input)

In [4]:
base_model_i = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model_i = add_preprocess(base_model_i, inception_v3.preprocess_input)

In [5]:
batches = image.ImageDataGenerator().flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_val = image.ImageDataGenerator().flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
batches_test = image.ImageDataGenerator().flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)
nb_batches_test = math.ceil(batches_test.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [6]:
# bf_val_x = model_x.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_x', bf_val_x)
bf_val_x = np.load(data_dir+'/results/bf_val_x.npy')

# bf_x_test = model_x.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_x_test', bf_x_test)
bf_x_test = np.load(data_dir+'/results/bf_x_test.npy')

In [18]:
# bf_val_i = model_i.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
# np.save(data_dir+'/results/bf_val_i', bf_val_i)
bf_val_i = np.load(data_dir+'/results/bf_val_i.npy')

# bf_i_test = model_i.predict_generator(batches_test, steps=nb_batches_test, verbose=1)
# np.save(data_dir+'/results/bf_i_test', bf_i_test)
bf_i_test = np.load(data_dir+'/results/bf_i_test.npy')

In [8]:
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               shear_range=0.15, 
                               zoom_range=0.1, 
                               channel_shift_range=10., 
                               horizontal_flip=True)

In [31]:
preds = []
nb_runs = 5
for i in range(nb_runs):
    print("i:", i)
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    y = to_categorical(batches.classes)
    bf_x = model_x.predict_generator(batches, steps=nb_batches, verbose=1)
    
    batches = gen.flow_from_directory(data_dir+'/train', target_size=target_size, batch_size=batch_size, shuffle=False)
    bf_i = model_i.predict_generator(batches, steps=nb_batches, verbose=1)
    
    lm = Sequential([Dense(120, activation='softmax', input_shape=(2048+2048,))])
    lm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
    lm.fit(np.hstack([bf_x, bf_i]), y, epochs=15, batch_size=nb_batches, 
           validation_data=(np.hstack([bf_val_x, bf_val_i]), y_val))
    
    pred = lm.predict(np.hstack([bf_x_test, bf_i_test]), batch_size=batch_size, verbose=1)
    preds.append(pred)

Found 8222 images belonging to 120 classes.
514/514 [==============================] - 124s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 113s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.0900 - acc: 0.6579 - val_loss: 0.7827 - val_acc: 0.8880
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5884 - acc: 0.8883 - val_loss: 0.4453 - val_acc: 0.9040
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3748 - acc: 0.9117 - val_loss: 0.3503 - val_acc: 0.9035
Epoch 4/15
8222/8222 [==============================] - 0s - loss: 0.2883 - acc: 0.9247 - val_loss: 0.3230 - val_acc: 0.9035
Epoch 5/15
8222/8222 [==============================] - 0s - loss: 0.2335 - acc: 0.9375 - val_loss: 0.3012 - val_acc: 0.9085
Epoch 6/15
8222/8222 [==============================] - 0s - loss: 0.1919 - acc: 0.9483 - val_loss: 0.2901 - val_acc: 0.9110
Epoch 7/15
8

8222/8222 [==============================] - 0s - loss: 0.0782 - acc: 0.9820 - val_loss: 0.2781 - val_acc: 0.9075
Epoch 13/15
8222/8222 [==============================] - 0s - loss: 0.0682 - acc: 0.9850 - val_loss: 0.2824 - val_acc: 0.9110
Epoch 14/15
8222/8222 [==============================] - 0s - loss: 0.0601 - acc: 0.9874 - val_loss: 0.2887 - val_acc: 0.9070
Epoch 15/15
514/514 [==============================] - 125s   
Found 8222 images belonging to 120 classes.
514/514 [==============================] - 112s   
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 0s - loss: 2.0829 - acc: 0.6546 - val_loss: 0.7996 - val_acc: 0.8630
Epoch 2/15
8222/8222 [==============================] - 0s - loss: 0.5855 - acc: 0.8905 - val_loss: 0.4512 - val_acc: 0.8990
Epoch 3/15
8222/8222 [==============================] - 0s - loss: 0.3701 - acc: 0.9146 - val_loss: 0.3611 - val_acc: 0.9045
Epoch 4/15
8222/8222 [==============================]

In [32]:
pred_ensemble = np.stack(preds).mean(axis=0)

### predict

In [33]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [34]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred_ensemble]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [35]:
description = 'xception_inception_ensemble_%d_data_aug' % nb_runs
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [36]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [37]:
!kg submit $submission_file_name -m $description

0.26989
